In [1]:
import os
import numpy as np
from scipy import stats
import pandas as pd
from sklearn.externals import joblib
from matplotlib import pyplot as plt

In [2]:
target = 'mnist'
#target = 'cifar10'
#start_epoch = 0
start_epoch = 19
end_epoch = 20
seed = 0

res = joblib.load('./%s/%s_%02d/eval_epoch_%02d_to_%02d.dat' % (target, target, seed, start_epoch, end_epoch))
res2 = joblib.load('./%s/%s_%02d/eval_epoch_%02d_to_%02d_outlier.dat' % (target, target, seed, start_epoch, end_epoch))
res.pop('baseline')
res2.pop('baseline')
methods = np.sort(np.unique([key[0] for key in res.keys()]))
methods2 = np.sort(np.unique([key[0] for key in res2.keys()]))
ks = np.sort(np.unique([key[1] for key in res.keys()]))
print(methods, methods2, ks)

['icml' 'random' 'sgd_all' 'sgd_last'] ['ae' 'iso'] [    1     3     6    10    30    60   100   300   600  1000  3000  6000
 10000]


In [3]:
n = 30

acc = np.zeros((ks.size, methods.size+methods2.size+1, n))
t = 5
for seed in range(n):
    try:
        res = joblib.load('./%s/%s_%02d/eval_epoch_%02d_to_%02d.dat' % (target, target, seed, start_epoch, end_epoch))
        res2 = joblib.load('./%s/%s_%02d/eval_epoch_%02d_to_%02d_outlier.dat' % (target, target, seed, start_epoch, end_epoch))
        acc[:, 0, seed] = res.pop('baseline')[t]
        for i, m in enumerate(methods):
            for j, k in enumerate(ks):
                acc[j, i+1, seed] = res[(m, k)][t]
        for i, m in enumerate(methods2):
            for j, k in enumerate(ks):
                acc[j, methods.size+i+1, seed] = res2[(m, k)][t]
    except:
        continue

idx = np.where(np.min(acc, axis=(0,1)))[0]
acc = acc[:, :, idx]
print(acc.shape)

(13, 7, 30)


In [ ]:
plt.semilogx(ks, 1-np.mean(acc, axis=2))
plt.legend(['baseline', *methods.tolist(), *methods2.tolist()])
plt.xlabel('# of instances removed')
plt.ylabel('missclassification rate')
if target == 'mnist':
    plt.ylim([0.007, 0.011])
elif target == 'cifar10':
    plt.ylim([0.155, 0.175])
plt.title('%s' % (target,))
plt.show()